#### SAMD51 Serial

In [1]:
import serial, time, pandas as pd, numpy as np, struct, pickle
from pathlib import Path
from typing import Literal, Dict, List, Optional
from rich import print

class ArduinoReader:
    def __init__(self, port="COM11", baudrate: int = 115200, timeout: float = 1.0):
        """
        port: e.g. 'COM11' on Windows, '/dev/ttyACM0' on Linux
        baudrate: match Arduino Serial.begin()
        timeout: read timeout in seconds
        """
        self.ser = serial.Serial(port, baudrate=baudrate, timeout=timeout)
        # give Arduino time to reset when serial opens
        self.data = {'rpm': [], 'sound': [], 'lift': []}
        time.sleep(1)

    def read_line(self) -> str:
        line = self.ser.readline().decode(errors="ignore")
        try:
            return list(map(float, line.strip().split()))    
        except:
            return line

    def get_data(self) -> Dict[str, np.ndarray]:
        CHUNK_SIZE = 1024
        HEADER_SIZE = 16 

        ser = self.ser
        ser.reset_input_buffer()
        ser.write(b' ')
        ser.flush()
        time.sleep(0.5)

        header = ser.read(HEADER_SIZE)
        if len(header) < HEADER_SIZE:
            raise RuntimeError(f"Bad or missing header: {header}, len: {len(header)}")
        
        header = np.frombuffer(header, np.uint32)
        if header[0] != 123456 and header[1] != 654321:
            raise RuntimeError(f"Header correct size, but sync string is wrong: {header}, len: {len(header)}")

        count = header[2]
        self.lift = header[3].astype(np.float32) / 1000
        sound = np.zeros(count, dtype=np.int16)
        light = np.zeros(count, dtype=np.int16)
        vibration = np.zeros(count, dtype=np.int16)
        buffers = [sound, light, vibration]

        while True:
            peek = ser.read(1)
            if not peek:
                break

            # END marker
            if peek == b'E':
                rest = ser.read(2)
                if rest == b'ND':
                    break
                continue

            # 7-byte packet header (1 + 6)
            rest = ser.read(6)
            if len(rest) < 6:
                break
            pkt_hdr = peek + rest

            tag, chunk_id, n, arr_idx, _ = struct.unpack('<BHHBB', pkt_hdr)
            if arr_idx > 2:
                continue

            payload = ser.read(n)
            if len(payload) != n:
                break

            start = chunk_id * (CHUNK_SIZE // 2)
            samples = len(payload) // 2
            end = start + samples
            buffers[arr_idx][start:end] = np.frombuffer(payload, dtype=np.uint16)

        self.df =pd.DataFrame({
            "sound": np.array(sound).astype(np.float32),
            "light": np.array(light).astype(np.float32),
            "vibration": np.array(vibration).astype(np.float32),
        })

    def tare_scale(self):
        self.ser.write(b't')
        self.ser.flush()

    def save_data(self, save_file: Path, cut_top_bottom=3, avg_cols=[('sound', 100), ('vibration', 100), ('light', 10)], extra_data={}):
        df = self.df
        for col, roll in avg_cols:

            for i in range(cut_top_bottom):
                index = df[col].argmax()
                if index > 0:
                    df.loc[index, col] = df.loc[index-1, col]
                else:
                    df.loc[index, col] = df.loc[index+1, col]

            for i in range(cut_top_bottom):
                index = df[col].argmin()
                if index > 0:
                    df.loc[index, col] = df.loc[index-1, col]
                else:
                    df.loc[index, col] = df.loc[index+1, col]

            df[col] = df[col].ffill().rolling(roll, center=True).mean()
        df.loc[len(df)] = [self.lift, 0, 0]

        with open(save_file, 'wb') as fp:
            pickle.dump({'lift': self.lift, 'arduino': df, **extra_data}, fp)

    def close(self):
        """Close serial connection."""
        self.ser.close()

    def get_weight(self):        
        ser = self.ser
        ser.reset_input_buffer()
        ser.write(b'w')
        ser.flush()
        return np.frombuffer(self.ser.read(4), np.float32)[0]

    def __repr__(self):
        return f"Lift: {self.data['lift']}"

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.close()


#### Rigol Serial

In [2]:
from time import perf_counter_ns, sleep
import numpy as np, plotly.express as px, pandas as pd, pyvisa
from typing import List, Dict, TypedDict, Optional, Callable


class RigolPSU():
    def __init__(self, resource_index=0):
        self.ar = ArduinoReader()
        self.ar.tare_scale()
        rm = pyvisa.ResourceManager()
        resources = rm.list_resources()
        if not resources:
            raise RuntimeError("No VISA instruments found.")
        self.instrument = rm.open_resource(resources[resource_index])

        # Set sensible defaults
        self.instrument.timeout = 5000
        self.instrument.read_termination = '\n'
        self.instrument.write_termination = '\n'

        self.on = False
        self.channel = 1
        self.data = {'voltage': [], 'power': []}

        # print("Connected to:", self.instrument.query("*IDN?").strip())

    def set_voltage(self, voltage: float, current_limit: float):
        self.instrument.write(f"APPL CH{self.channel},{voltage},{current_limit}")
        if not self.on:
            self.instrument.write(f"OUTP CH{self.channel},ON")
            self.on = True

    def measure_power(self) -> float:
        voltage = float(self.instrument.query(f"MEAS:VOLT? CH{self.channel}"))
        current = float(self.instrument.query(f"MEAS:CURR? CH{self.channel}"))
        return round(voltage * current, 2)
    
    def measure_power_over_time(self, seconds: float, time_step=.05):
        start = perf_counter_ns()
        end = start + seconds * 1e9
        time = []
        power = []
        while (current_time := perf_counter_ns()) < end:
            time.append(current_time)
            power.append(self.measure_power())
            sleep(time_step)
        return dict(x=(np.array(time)-start)/1e9, y=np.array(power))
    
    def volt_sweep(self, tot_measurements=10, min_voltage = 2, max_voltage=10.0, current_limit: float = 2.0, data_folder=Path('data')):

        power_measurements = []
        voltages = np.linspace(min_voltage, max_voltage, tot_measurements).round(1)
        used_voltages = []
        for voltage in voltages:
            self.ar.get_data()
            self.ar.save_data(data_folder / f"{voltage:.2f}_arduino", extra_data={'power': self.measure_power(), 'voltage': voltage})
            self.set_voltage(voltage, current_limit)
            used_voltages.append(voltage)
            power_measurements.append(self.measure_power())

        self.data = dict(voltage=used_voltages, power=np.array(power_measurements))
    
    def output_on(self):
        self.instrument.write(f"OUTP CH{self.channel},ON")
        self.on = True

    def output_off(self):
        self.instrument.write(f"OUTP CH{self.channel},OFF")
        self.on = False

    def close(self):
        if self.on:
            self.output_off()
        self.instrument.close()

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.close()




In [ ]:
# sometimes the computer wont conect to the arduino.
# there needs to be a system to handle this initial connection. 
# but when it works, it works.

with ArduinoReader() as ar:
    ar.tare_scale()
    print(ar.get_weight())

IndexError: index 0 is out of bounds for axis 0 with size 0

In [7]:
with RigolPSU() as psu:
    psu.volt_sweep()
del psu

SerialException: could not open port 'COM11': PermissionError(13, 'Access is denied.', None, 5)

In [122]:
ar.save_data('tmp')

In [ ]:


df2 = clean_cols(df.copy())
px.line(df2.vibration)


In [ ]:
from typing import TypedDict
import pandas as pd

class HoverPower(TypedDict):
    thrust_N: pd.Series
    v_induced_mps: pd.Series
    power_ideal_W: pd.Series
    efficiency: pd.Series

def prop_hover_power(
    grams_lift: pd.Series,
    power_in: pd.Series,
    air_density=1.05,
    diameter_m = 0.10):
    """
    Compute ideal hover power for a prop using momentum theory.
    Args:
        grams_lift: Desired lift in grams.
        air_density: Air density in kg/m^3 (e.g., ~1.05 for Calgary ~22.5 °C).
        diameter_m: Prop diameter in meters (default 0.10 m).
    """

    G = 9.80665
    # Convert lift to thrust
    thrust_N = (grams_lift / 1000.0) * G
    radius = diameter_m / 2.0
    area_m2 = 3.141592653589793 * radius * radius
    v_induced = (thrust_N / (2.0 * air_density * area_m2)) ** 0.5
    power_ideal = thrust_N * v_induced
    efficiency = power_ideal / power_in

    return pd.DataFrame(HoverPower(
        thrust_N= thrust_N,
        v_induced_mps= v_induced,
        power_ideal_W= power_ideal,
        efficiency=efficiency
    ))


In [ ]:
import os
import pandas as pd
import plotly.express as px
import plotly.io as pio

cols = Literal['rpm', 'sound', 'lift', 'voltage', 'power', 'lift_per_weight']

def load_dataframes_from_folder(folder: str):
    """Load all pickled DataFrames from a folder into a dict {name: DataFrame}."""
    dfs = {}
    for fname in os.listdir(folder):
        fpath = os.path.join(folder, fname)
        if os.path.isfile(fpath) and not fname.startswith("."):
            try:
                df: pd.DataFrame = pd.read_pickle(fpath)
                df = df.join(prop_hover_power(df.lift, df.power))
                dfs[fname] = df
            except Exception as e:
                print(f"Skipping {fname} (not a valid pickle): {e}")
    return dfs

def multi_scatter(dfs: dict, x: str, y: str, title: str, height=490):
    """
    Combine multiple DataFrames into one scatter plot,
    styled for a bright/white background.
    """
    # Collect all data with "source" column
    frames = []
    for label, df in dfs.items():
        d = df.copy()
        d["source"] = label
        frames.append(d)
    df_all = pd.concat(frames)

    # Build scatter
    fig = px.scatter(
        df_all,
        x=x,
        y=y,
        color="source",
        title=title,
        labels={x: x.capitalize(), y: y.capitalize(), "source": "Dataset"},
        opacity=0.85,
        symbol="source"
    )

    # Style for bright background
    fig.update_traces(
        marker=dict(size=9, line=dict(width=1, color="black"))
    )

    fig.update_layout(
        template="simple_white",  # bright, clean background
        autosize=True,
        height=height,
        title=dict(
            text=f"<b>{title}</b>",
            font=dict(size=26, color="black"),
            x=0.5,
            xanchor="center"
        ),
        legend=dict(
            title="<b>Datasets</b>",
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="black",
            borderwidth=1,
            font=dict(size=13, color="black")
        ),
        margin=dict(l=40, r=40, t=80, b=40),
        xaxis=dict(
            showline=True,          # draw axis line
            linecolor="black",      # axis line color
            linewidth=2,            # axis line thickness
            showgrid=True,          # show background grid
            gridcolor="rgba(0,0,0,0.1)", 
            showticklabels=True,    # display tick labels
            ticks="outside",        # ticks outside the plot
            tickcolor="black",      # tick color
            ticklen=6,              # tick length
            tickwidth=2,            # tick thickness
            zeroline=False,         # no zero-line overlay
            title=dict(text=x.capitalize(), font=dict(size=16, color="black")),
            tickfont=dict(size=13, color="black")
        ),
        yaxis=dict(
            showline=True,
            linecolor="black",
            linewidth=2,
            showgrid=True,
            gridcolor="rgba(0,0,0,0.1)",
            showticklabels=True,
            ticks="outside",
            tickcolor="black",
            ticklen=6,
            tickwidth=2,
            zeroline=False,
            title=dict(text=y.capitalize(), font=dict(size=16, color="black")),
            tickfont=dict(size=13, color="black")
        ),

        plot_bgcolor="white",
        paper_bgcolor="white"
    )

    return fig


# === usage ===
dfs = load_dataframes_from_folder("data")
figs = []

figs.append(multi_scatter(dfs, "power", "efficiency", "Power In vs Propeller Efficiency"))
figs.append(multi_scatter(dfs, "rpm", "efficiency", "RPM vs Propeller Efficiency"))
figs.append(multi_scatter(dfs, "rpm", "lift_per_weight", "RPM vs Lift Per Propeller Weight"))
figs.append(multi_scatter(dfs, "power", "power_ideal_W", "Power In vs Power Out"))
figs.append(multi_scatter(dfs, "rpm",   "lift",           "RPM vs Lift"))
figs.append(multi_scatter(dfs, "rpm",   "sound",           "RPM vs Sound"))
figs.append(multi_scatter(dfs, "power", "lift_per_weight", "Power vs Eff"))
figs.append(multi_scatter(dfs, "power", "lift",           "Power vs Lift"))
figs.append(multi_scatter(dfs, "power", "rpm",           "Power vs RPM"))
figs.append(multi_scatter(dfs, "power", "sound",          "Power vs Sound"))
figs.append(multi_scatter(dfs, "voltage", "rpm",          "Volt vs RPM"))


with open("scatter_dashboard.html", "w") as f:
    f.write(pio.to_html(figs[0], include_plotlyjs='cdn', full_html=False))
    for fig in figs[1:]:
        f.write(pio.to_html(fig, include_plotlyjs=False, full_html=False))


In [ ]:
print(dfs['bent wing v1'].columns.to_list())